In [1]:
import numpy as np
from tqdm import tqdm 
import torch
import numpy as np
import random
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append("D:\\1st\\BPs_BPTK\\Python") 
from BPS_init_function_MultiParas import BPS_BPTK_MultiParas
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False # 用来正常显示负号
from sklearn.metrics import r2_score
import scipy.stats as stats
import os
os.chdir('d:\\1st\\BPs_BPTK')

In [2]:
# 定义神经网络模型
class ResNetBlock(nn.Module):
    def __init__(self, hyperparas):
        super(ResNetBlock, self).__init__()
        
        self.hidden_dim = hyperparas['hidden_dim']
        self.block_layer_nums =hyperparas['block_layer_nums']
            
        # Define layers for the function f (MLP)
        self.layers = nn.ModuleList()
        
        for _ in range(self.block_layer_nums - 1):  # -2 because we already added one layer and last layer is already defined
            self.layers.append(nn.Linear(self.hidden_dim,self.hidden_dim ))
        
        # Layer normalization
        self.layernorms = nn.ModuleList()
        for _ in range(self.block_layer_nums - 1):  # -1 because layer normalization is not applied to the last layer
            self.layernorms.append(nn.LayerNorm(self.hidden_dim))
        
    def forward(self, x):
        # Forward pass through the function f (MLP)
        out = x
        for i in range(self.block_layer_nums - 1):  # -1 because last layer is already applied outside the loop
            out = self.layers[i](out)
            out = self.layernorms[i](out)
            out = torch.relu(out)
        
        # Element-wise addition of input x and output of function f(x)
        out = x + out
        
        return out
    

class CustomResNN(nn.Module):
    def __init__(self,hyperparas):
        super().__init__()
        self.input_dim = hyperparas['input_dim'] #68
        self.hidden_dim = hyperparas['hidden_dim'] #30
        self.hidden_nums = hyperparas['hidden_nums'] #3
        self.output_dim = hyperparas['output_dim'] #3
        self.block_layer_nums = hyperparas['block_layer_nums'] #3

        self.layer_list = []
        self.layer_list.append(nn.Sequential(nn.Linear(self.input_dim,self.hidden_dim),nn.ReLU() ) )

        for _ in range(self.hidden_nums-1):
            self.layer_list.append(ResNetBlock(hyperparas))

        self.layer_list.append(nn.Linear(self.hidden_dim,self.output_dim))

        self.linear_Res_final = nn.Sequential(*self.layer_list)

    def forward(self,inputs):
        
        return self.linear_Res_final(inputs)

#超参数合集
hyperparas = {'input_dim':15,'hidden_dim':30,'hidden_nums':3,'output_dim':3,'block_layer_nums':3}
learning_rate = 0.001
num_epochs = 100

bestmodel = CustomResNN(hyperparas)

In [3]:
best_model_path ='Python\optim\Temporary_Model\model_pause1.pth'
#best_model_path ='Python\optim\Temporary_Model\model_pause2.pth'
#best_model_path ='Python\optim\Temporary_Model\model_pause3.pth'
best_model_path ='Python\optim\Temporary_Model\model_best.pth'

#best_model_path ='Python\optim\Settled_Model\Dual5plus10\model1.pth'


# 准备数据
X1 = np.load("Python\optim\DataFromBPTK\\plasma5_add.npy")  #输入数据
X2 = np.load("Python\optim\DataFromBPTK\\urinebps5_1_add.npy")  #输入数据
X3 = np.load("Python\optim\DataFromBPTK\\urinebpsg5_1_add.npy")  #输入数据
X = np.hstack((X1,X2,X3))
print(np.shape(X))

Data_origin = X

sampling_time_index = np.array([98,433,601,1644,3651])
time_range = sampling_time_index/200 #采样时间节点，在0至75小时内共选取了5个时间节点 

sampling_time_index = np.array([101,433,608,1653,3661])
time_range1 = sampling_time_index/200 #采样时间节点，在0至75小时内共选取了5个时间节点 

#paras = np.array([[17.28, 6.39, 5.7],[14.91,2.78,4.707],[21.333, 9.666, 7.42],[19.921, 11, 2.4212],[23.032,7.2223,4.99],[13.53,10.11,5.444],[18.888,9.999,8.888],[15.5,10.5,7.5],[12.45,6.84,6.75],[20.8,4.61,9.888]])
time = np.arange(0,75,0.005)

(47743, 15)


In [10]:
#宽距取样
mean1 = 17.28
mean2 = 6.39
mean3 = 5.7
X1 = stats.truncnorm(-2, 2.3, loc=mean1, scale=5)
x1 = X1.rvs(size = 6,random_state = 43)
X2 = stats.truncnorm(-1, 1.3, loc=mean2, scale=5)
x2 = X2.rvs(size = 5,random_state = 43)
X3 = stats.truncnorm(-1, 1.2, loc=mean3, scale=5)
x3 = X3.rvs(size = 5,random_state = 43)
paras = np.array(list(itertools.product(x1, x2, x3)))

In [8]:
#窄距取样
mean1 = 17.28
mean2 = 6.39
mean3 = 5.7
X1 = stats.truncnorm(-2, 2, loc=mean1, scale=3)
x1 = X1.rvs(size = 6,random_state = 43)
X2 = stats.truncnorm(-1, 1.2, loc=mean2, scale=2)
x2 = X2.rvs(size = 5,random_state = 43)
X3 = stats.truncnorm(-1, 1.2, loc=mean3, scale=2)
x3 = X3.rvs(size = 5,random_state = 43)
paras = np.array(list(itertools.product(x1, x2, x3)))

In [6]:
def test_NewData_NN(origin_para = paras , model = bestmodel, model_path = best_model_path, Data = Data_origin, sampling_time_range_plasma = time_range, sampling_time_range_urine = time_range1):

    scaler = StandardScaler()
    X = scaler.fit_transform(Data)
    best_model = model
    best_model.load_state_dict(torch.load(model_path))
    id = 0
    time = np.arange(0,75,0.005)
    sampling_time_index_plasma = (200*sampling_time_range_plasma).astype(int) #采样时间节点在求解器结果中的索引值
    sampling_time_index_urine = (200*sampling_time_range_urine).astype(int) #采样时间节点在求解器结果中的索引值

   
    norm1_error = np.zeros((np.shape(origin_para)[0],2))
    r2 = np.zeros((np.shape(origin_para)[0],1)).flatten()

    plasma_true,urine_true,urineg_true =BPS_BPTK_MultiParas(t = time,volunteer_ID =id, paras = origin_para ,mode = '63')

    X_test = np.hstack((plasma_true[:,sampling_time_index_plasma],urine_true[:,sampling_time_index_urine],urineg_true[:,sampling_time_index_urine]))
    y_test = origin_para

    X_test = scaler.transform(X_test)

    
    X_test = torch.tensor(X_test).float()
    y_test = torch.tensor(y_test).float()

    test_outputs = best_model(X_test)
    example_FromNN_3para = test_outputs
    label_rel_err = test_outputs/y_test-1
    label_rel_err_mean = torch.sum(label_rel_err,dim=0)/np.shape(origin_para)[0]

    
    example_FromNN_3para=example_FromNN_3para.detach().numpy()
    
    plasma_fromNN,urine_fromNN,urineg_fromNN = BPS_BPTK_MultiParas(t = time,volunteer_ID =id, paras = example_FromNN_3para , mode = '63')

    
    err28plus30 = np.zeros((1,2)).flatten()
    slice_true = np.hstack((plasma_true[:,sampling_time_index_plasma],urine_true[:,sampling_time_index_urine],urineg_true[:,sampling_time_index_urine]))
    slice_fromNN = np.hstack((plasma_fromNN[:,sampling_time_index_plasma],urine_fromNN[:,sampling_time_index_urine],urineg_fromNN[:,sampling_time_index_urine]))
    for i in range(np.shape(origin_para)[0]):
        
        r2[i] = r2_score(plasma_true[i,:], plasma_fromNN[i,:]) #决定系数
        norm_absolute = np.linalg.norm(plasma_fromNN[i,:]-plasma_true[i,:], ord=1)/15000
        norm_relative = np.linalg.norm(plasma_fromNN[i,1:]/(plasma_true[i,1:]+1E-16)-1, ord=1)/14999
        #print(norm_relative)
        norm1_error[i,0] = norm_absolute
        norm1_error[i,1] = norm_relative
                
        
    err28plus30[0] = np.mean(abs(slice_fromNN-slice_true))
    err28plus30[1] = np.mean(abs(slice_fromNN-slice_true)/(slice_true+1E-11))   
    mean_r2 = np.mean(r2)#平均决定系数



    return err28plus30,label_rel_err_mean, mean_r2

In [7]:
err28plus30,label_rel_err_mean,mean_r2 = test_NewData_NN(origin_para = paras , model = bestmodel, model_path = best_model_path, Data = Data_origin, sampling_time_range_plasma = time_range, sampling_time_range_urine = time_range1)

label_rel_err_mean = label_rel_err_mean.detach().numpy()
print(f'{label_rel_err_mean}  {np.mean(abs(label_rel_err_mean))}   标签MRE')
print(f'{err28plus30}    MAE&MRE切片')
print(f'{mean_r2}    R^2')

  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [00:21<00:00,  6.82it/s]


[ 0.00339488 -0.00210755  0.00387542]  0.0031259479001164436   标签MRE
[0.00013728 0.00717461]    MAE&MRE切片
0.9988622611142004    R^2


In [11]:
err28plus30,label_rel_err_mean,mean_r2 = test_NewData_NN(origin_para = paras , model = bestmodel, model_path = best_model_path, Data = Data_origin, sampling_time_range_plasma = time_range, sampling_time_range_urine = time_range1)

label_rel_err_mean = label_rel_err_mean.detach().numpy()
print(f'{label_rel_err_mean}  {np.mean(abs(label_rel_err_mean))}   标签MRE')
print(f'{err28plus30}    MAE&MRE切片')
print(f'{mean_r2}    R^2')

  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [00:21<00:00,  6.88it/s]


[0.02203444 0.00732104 0.04299994]  0.024118473753333092   标签MRE
[0.00032513 0.02396196]    MAE&MRE切片
0.9787524296167548    R^2
